# Investigate primary key issues in GTFS schedule data

October 26-31, 2022 / Laurie 

See [this Google Doc](https://docs.google.com/document/d/12OtxayJRUMWO9HrJS11c6iiuw4OHPCgJv01YtnioZxY/edit#) for more info.

In [3]:
import pandas as pd
import pandas_gbq
import base64
import datetime

# trips

First, ran `poetry run dbt run -s +dim_trips` to refresh data in my namespace. 

TLDR:

* Victor Valley GMV and Merced GMV are the only current feeds in the v2 pipeline that have duplicate `trip_id` values within individual feeds.
* For Merced GMV:
   * They are all cases where the key is duplicated but the data is actually different (i.e., worst case scenario).
   * 2/412 (.4%) trips in their feed are affected. Same `trip_id`, different `route_` and `shape_` ids. 
* For Victor Valley GMV: 
   * At the start of v2 data availability (9/15/22), there was a mixture of cases where the key was duplicated with identical and different data (i.e., there are trip IDs that appear at least 3 times: 2 identical duplicates + another row with the same key but different data.) At that time, 46/1307 (3.5%) trips were duplicates of some kind, with most being full duplicates where the entire row was replicated exactly.
   * In the most recent feed (10/25/22), there are **only** full duplicates where the entire row is identical (this is the best case scenario, because these are safe to deduplicate). Incidence has decreased to 26/1838 (1.4%). 

In [ ]:
# this is the query from the failing dbt test
# combined with left joining back on to the full table to get the full trip details for the dup rows
raw_dup_trips  = pd.read_gbq(
    """
    with validation_errors as (

        select
            base64_url, ts, trip_id
        from `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__trips`
        group by base64_url, ts, trip_id
        having count(*) > 1

    ),
    
    erroring_trips as (
    
        SELECT 
          t2.*
        FROM validation_errors t1 
        LEFT JOIN `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__trips` t2
        USING(trip_id, base64_url, ts)
    
    )

    select *
    from erroring_trips
    """, project_id = 'cal-itp-data-infra-staging')

In [ ]:
dup_trips = raw_dup_trips.copy()

In [ ]:
len(dup_trips)

In [ ]:
# drop totally empty columns because they mess up joins later
dup_trips.dropna(axis = 1, how = "all", inplace = True)

In [ ]:
# how many feeds affected?
dup_trips.base64_url.unique()

In [ ]:
# convert base64_url to string 
dup_trips['url'] = dup_trips['base64_url'].apply(lambda x: str(base64.urlsafe_b64decode(x), encoding = 'utf-8'))

In [ ]:
# which urls (human readable) are affected?
dup_trips.url.unique()

In [ ]:
# how many are cases where ALL values are identical?
orig_trip_cols = list(dup_trips.columns)

In [ ]:
dup_trips['dummy'] = 1
# notebooks are annoying just in case we run out of order
if 'dummy' in orig_trip_cols:
    orig_trip_cols.remove('dummy')
summarize_full_trip_dups = dup_trips.groupby(by = orig_trip_cols, dropna = False)['dummy'].sum().reset_index().rename(columns = {'dummy': 'ct'})

In [ ]:
# cases where whole row is identical
entire_trip_dups = summarize_full_trip_dups.loc[summarize_full_trip_dups.ct > 1, :].copy()
entire_trip_dups['full_dup'] = True

In [ ]:
# how many cases are covered by entire dups?
check_trip_entire_dups = dup_trips.merge(entire_trip_dups, how = 'left', on = orig_trip_cols, suffixes = ['', '_ent_dup'])

In [ ]:
# check for issue type by URL
check_trip_entire_dups['full_dup'] = check_trip_entire_dups['full_dup'].fillna(False)

In [ ]:
# check for cases where it's not a full dup
check_trip_entire_dups[~check_trip_entire_dups.full_dup].url.unique()

In [ ]:
# check for cases where it is a full dup
check_trip_entire_dups[check_trip_entire_dups.full_dup].url.unique()

In [ ]:
# check at trip id level for whether all instances of trip id are full dup
trip_id_entire_dups = check_trip_entire_dups.groupby(['url', 'trip_id', 'ts', 'full_dup']).size().reset_index()

In [ ]:
# check for mixed cases within a trip id 
trip_id_entire_dups = trip_id_entire_dups.groupby(['url', 'trip_id', 'ts']).agg({'full_dup': ['any', 'all']}).reset_index()
trip_id_entire_dups.columns = ['url', 'trip_id', 'ts', 'any_full_dups', 'all_full_dups']

In [ ]:
# flag cases where there are mix of fully same and not 
trip_id_entire_dups['mixed_cases'] = trip_id_entire_dups.any_full_dups != trip_id_entire_dups.all_full_dups

In [ ]:
trip_id_entire_dups[trip_id_entire_dups.mixed_cases].url.unique()

In [ ]:
# add the dup type flags to each individual row
check_trip_entire_dups = check_trip_entire_dups.merge(trip_id_entire_dups, how = 'left', on = ['url', 'trip_id', 'ts'])

In [ ]:
# summarize typology of dups
entire_dup_trips_pivot = check_trip_entire_dups.groupby(['url', 'ts', 'mixed_cases', 'any_full_dups', 'all_full_dups'])['trip_id'].count().reset_index()

In [ ]:
# read in all trips for affected feeds
all_vv_merced_trips_raw = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__trips`
    where base64_url = 'aHR0cHM6Ly9vbnRpbWUudnZ0YS5vcmcvZ3Rmcw==' or base64_url = 'aHR0cHM6Ly90aGVidXNsaXZlLmNvbS9ndGZz'
    """, project_id = 'cal-itp-data-infra-staging')

In [ ]:
all_vv_merced_trips = all_vv_merced_trips_raw.copy()

In [ ]:
# convert base64_url to string 
all_vv_merced_trips['url'] = all_vv_merced_trips['base64_url'].apply(lambda x: str(base64.urlsafe_b64decode(x), encoding = 'utf-8'))

In [ ]:
# summarize trip count per feed
num_trips = all_vv_merced_trips.groupby(['url', 'ts'])['trip_id'].count().reset_index().rename(columns = {'trip_id': 'num_trips'})

In [ ]:
entire_dup_trips_pivot = entire_dup_trips_pivot.pivot(index = ['url', 'ts'], columns = ['mixed_cases', 'any_full_dups', 'all_full_dups'], values = 'trip_id').reset_index()

In [ ]:
entire_dup_trips_pivot.columns = ['url', 'ts', 'ct_no_full_dups', 'ct_all_full_dups', 'ct_mix_full_and_partial_dups']

In [ ]:
entire_dup_trips_pivot['tot_dups'] = entire_dup_trips_pivot.ct_no_full_dups.fillna(0) + entire_dup_trips_pivot.ct_all_full_dups.fillna(0) + entire_dup_trips_pivot.ct_mix_full_and_partial_dups.fillna(0)

In [ ]:
dup_trips_summary = entire_dup_trips_pivot.merge(num_trips, how = 'outer', on = ['url', 'ts'])

In [ ]:
dup_trips_summary['dup_prop'] = round((dup_trips_summary.tot_dups / dup_trips_summary.num_trips),4)

In [ ]:
dup_trips_summary.dtypes

In [ ]:
dup_trips_summary[(dup_trips_summary.ts.dt.date == datetime.date(2022, 10, 25)) | (dup_trips_summary.ts.dt.date == datetime.date(2022, 9, 15))]

In [ ]:
dup_trips[(dup_trips.url == 'https://thebuslive.com/gtfs') & (dup_trips.ts.dt.date == datetime.date(2022, 10, 25))]

# stops & stop_times

First, ran `poetry run dbt run -s +dim_stops +dim_stop_times` to refresh data in my namespace & `poetry run dbt test -s stg_gtfs_schedule__stops stg_gtfs_schedule__stop_times dim_stops dim_stop_times` to identify failing tests.

There appear to be several distinct issues in these two files, will need to address separately and together.

* Affects both: null `stop_id` values in both files -- need to check if these are related (same feed) or distinct
* `stops` also has duplicate non-null `stop_id` values
* `stop_times` has duplicate primary keys (`trip_id` + `stop_sequence`)

## null `stop_id`s

Confirmed that missing `stop_id` values in both `stops` and `stop_times` are due to `Tuolumne Schedule` only, predominanly flag / on-demand stops.

In [4]:
raw_stops_null_stop_ids  = pd.read_gbq(
    """
    SELECT 
        *
    FROM `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__stops`
    WHERE stop_id IS NULL
    """, project_id = 'cal-itp-data-infra-staging')

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
len(raw_stops_null_stop_ids)

1080

In [6]:
raw_stops_null_stop_ids.base64_url.unique()

array(['aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0ZnMtZGF0YS9zY2hlZHVsZS1hc3NldHMvdGN0YS0yMDIxLTEyLTEwLnppcA=='],
      dtype=object)

In [7]:
raw_stop_times_null_stop_ids  = pd.read_gbq(
    """
    SELECT 
        *
    FROM `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__stop_times`
    WHERE stop_id IS NULL
    """, project_id = 'cal-itp-data-infra-staging')

In [8]:
len(raw_stop_times_null_stop_ids)

8802

In [9]:
raw_stop_times_null_stop_ids.base64_url.unique()

array(['aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0ZnMtZGF0YS9zY2hlZHVsZS1hc3NldHMvdGN0YS0yMDIxLTEyLTEwLnppcA=='],
      dtype=object)

In [10]:
tuolumne_stops  = pd.read_gbq(
    """
    SELECT 
        *
    FROM `cal-itp-data-infra-staging`.`laurie_staging`.`stg_gtfs_schedule__stops`
    WHERE base64_url = 'aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0ZnMtZGF0YS9zY2hlZHVsZS1hc3NldHMvdGN0YS0yMDIxLTEyLTEwLnppcA=='
    """, project_id = 'cal-itp-data-infra-staging')

In [11]:
len(tuolumne_stops)

4050

In [18]:
stop_cols_to_dedupe = list(tuolumne_stops.columns)
stop_cols_to_dedupe.remove('ts')

In [20]:
tuolumne_stops.drop_duplicates(subset = stop_cols_to_dedupe)

,base64_url,ts,stop_id,tts_stop_name,stop_lat,stop_lon,zone_id,parent_station,stop_code,stop_name,stop_desc,stop_url,location_type,stop_timezone,wheelchair_boarding,level_id,platform_code
0,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,38.067467,-120.188136,None,None,None,Mi-Wuk Village - Sweet peas (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
54,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,38.015511,-120.396417,None,None,None,Parrots Ferry @ Union Hill,None,None,<NA>,None,<NA>,None,None
108,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,001,None,37.972087,-120.366040,None,None,None,Crossroads Shopping Center (Walmart),None,None,<NA>,None,<NA>,None,None
162,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,38.024024,-120.401611,None,None,None,Parrotts Ferry @ Gold Rush MHP (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
216,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,510,None,37.974195,-120.239537,None,None,None,Black Oak Casino Resort,None,None,<NA>,None,<NA>,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3780,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,37.982027,-120.382007,None,None,None,Washington/Church St (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
3834,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,240,None,37.977329,-120.342252,None,None,None,Mono Way @ Fuel Depot,None,None,<NA>,None,<NA>,None,None
3888,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,37.974160,-120.370938,None,None,None,Sonora Plaza (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
3942,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-10-27 03:00:15.695291+00:00,None,None,37.977683,-120.324835,None,None,None,Mono Way @ Indian Rock Center (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None


In [21]:
raw_stops_null_stop_ids.drop_duplicates(subset = stop_cols_to_dedupe)

,base64_url,ts,stop_id,tts_stop_name,stop_lat,stop_lon,zone_id,parent_station,stop_code,stop_name,stop_desc,stop_url,location_type,stop_timezone,wheelchair_boarding,level_id,platform_code
0,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,38.067467,-120.188136,None,None,None,Mi-Wuk Village - Sweet peas (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
1,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,38.061422,-120.199348,None,None,None,Sugar Pine (Fire Department) (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
2,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,37.976985,-120.321444,None,None,None,Mono Village Shopping Center @ Mikes Pizza (Fl...,Flag Stop,None,<NA>,None,<NA>,None,None
3,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,37.977683,-120.324835,None,None,None,Mono Way @ Indian Rock Center (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
4,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,38.038330,-120.377813,None,None,None,Yankee Hill / Big Hill (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
5,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,38.024024,-120.401611,None,None,None,Parrotts Ferry @ Gold Rush MHP (Flag Stop),Flag Stop,None,<NA>,None,<NA>,None,None
6,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,38.027355,-120.410116,None,None,None,Columbia Airport (On-Demand),On-Demand Stop. Please call 209-532-0404 to re...,None,<NA>,None,<NA>,None,None
7,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,37.986977,-120.370954,None,None,None,Blackberry Oaks (On-Demand),On-Demand Stop. Please call 209-532-0404 to re...,None,<NA>,None,<NA>,None,None
8,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,37.977079,-120.343032,None,None,None,Mono Way @ Elsey Ct (serving the Junction) (On...,On-Demand Stop. Please call 209-532-0404 to re...,None,<NA>,None,<NA>,None,None
9,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-16 14:35:07.584536+00:00,None,None,37.990160,-120.384922,None,None,None,Sonora High School (On-Demand),On-Demand Stop. Please call 209-532-0404 to re...,None,<NA>,None,<NA>,None,None


In [24]:
stop_times_cols_to_dedupe = list(raw_stop_times_null_stop_ids.columns)
stop_times_cols_to_dedupe.remove('ts')

In [26]:
raw_stop_times_null_stop_ids.drop_duplicates(subset = stop_times_cols_to_dedupe).sort_values(by = ['trip_id', 'stop_sequence'])

,base64_url,ts,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint
5886,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,10,None,2,07:36:00,07:36:00,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
0,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,10,None,3,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
5994,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,10,None,5,07:59:00,07:59:00,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
6642,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,11,None,2,10:36:00,10:36:00,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
1,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,11,None,3,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,92,None,8,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
102,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,93,None,6,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
103,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,93,None,8,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
104,aHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2d0Zn...,2022-09-15 14:59:54.274779+00:00,94,None,6,None,None,None,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
